<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/workflow/human_in_the_loop_story_crafting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Choose Your Own Adventure Workflow (Human In The Loop)

For some Workflow applications, it may desirable and/or required to have humans involved in its execution. For example, a step of a Workflow may need human expertise or input in order to run. In another scenario, it may be required to have a human validate the initial output of a Workflow.

In this notebook, we show how one can implement a human-in-the-loop pattern with Workflows. Here we'll build a Workflow that creates stories in the style of Choose Your Own Adventure, where the LLM produces a segment of the story along with potential actions, and a human is required to choose from one of those actions.

## Generating Segments Of The Story With An LLM

Here, we'll make use of the ability to produce structured outputs from an LLM. We will task the LLM to create a segment of the story that is in continuation of previously generated segments and action choices.

In [3]:
# install the required packages
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.1 MB/s eta 0:00:00


In [4]:
from typing import Any, List

from llama_index.llms.openai import OpenAI
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.prompts import PromptTemplate

In [5]:
class Segment(BaseModel):
    """Data model for generating segments of a story."""

    plot: str = Field(
        description="The plot of the adventure for the current segment. The plot should be no longer than 3 sentences."
    )
    actions: List[str] = Field(
        default=[],
        description="The list of actions the protaganist can take that will shape the plot and actions of the next segment.",
    )

In [6]:
SEGMENT_GENERATION_TEMPLATE = """
You are working with a human to create a story in the style of choose your own adventure.

The human is playing the role of the protaganist in the story which you are tasked to
help write. To create the story, we do it in steps, where each step produces a BLOCK.
Each BLOCK consists of a PLOT, a set of ACTIONS that the protaganist can take, and the
chosen ACTION.

Below we attach the history of the adventure so far.

PREVIOUS BLOCKS:
---
{running_story}

Continue the story by generating the next block's PLOT and set of ACTIONs. If there are
no previous BLOCKs, start an interesting brand new story. Give the protaganist a name and an
interesting challenge to solve.


Use the provided data model to structure your output.
"""

In [7]:
FINAL_SEGMENT_GENERATION_TEMPLATE = """
You are working with a human to create a story in the style of choose your own adventure.

The human is playing the role of the protaganist in the story which you are tasked to
help write. To create the story, we do it in steps, where each step produces a BLOCK.
Each BLOCK consists of a PLOT, a set of ACTIONS that the protaganist can take, and the
chosen ACTION. Below we attach the history of the adventure so far.

PREVIOUS BLOCKS:
---
{running_story}

The story is now coming to an end. With the previous blocks, wrap up the story with a
closing PLOT. Since it is a closing plot, DO NOT GENERATE a new set of actions.

Use the provided data model to structure your output.
"""

In [10]:
# set OPENAI_API_KEY from Colab Secrets
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [12]:
# Let's see an example segment
llm = OpenAI("gpt-4o-mini")
segment = llm.structured_predict(
    Segment,
    PromptTemplate(SEGMENT_GENERATION_TEMPLATE),
    running_story="",
)

In [13]:
segment

Segment(plot='In the heart of the enchanted forest, Elara, a skilled herbalist, discovers a hidden glade filled with glowing flowers. As she approaches, she hears whispers that seem to come from the flowers themselves, revealing a secret about a lost treasure hidden deep within the forest. However, a dark shadow looms nearby, threatening to guard the treasure and keep it hidden forever.', actions=['Investigate the glowing flowers', 'Follow the whispers deeper into the forest', 'Set a trap for the shadow', 'Leave the glade and return home'])

### Stitching together previous segments

We need to stich together story segments and pass this in to the prompt as the value for `running_story`. We define a `Block` data class that holds the `Segment` as well as the `choice` of action.

In [14]:
import uuid
from typing import Optional

BLOCK_TEMPLATE = """
BLOCK
===
PLOT: {plot}
ACTIONS: {actions}
CHOICE: {choice}
"""


class Block(BaseModel):
    id_: str = Field(default_factory=lambda: str(uuid.uuid4()))
    segment: Segment
    choice: Optional[str] = None
    block_template: str = BLOCK_TEMPLATE

    def __str__(self):
        return self.block_template.format(
            plot=self.segment.plot,
            actions=", ".join(self.segment.actions),
            choice=self.choice or "",
        )

In [15]:
block = Block(segment=segment)
print(block)


BLOCK
===
PLOT: In the heart of the enchanted forest, Elara, a skilled herbalist, discovers a hidden glade filled with glowing flowers. As she approaches, she hears whispers that seem to come from the flowers themselves, revealing a secret about a lost treasure hidden deep within the forest. However, a dark shadow looms nearby, threatening to guard the treasure and keep it hidden forever.
ACTIONS: Investigate the glowing flowers, Follow the whispers deeper into the forest, Set a trap for the shadow, Leave the glade and return home
CHOICE: 



## Create The Choose Your Own Adventure Workflow

This Workflow will consist of two steps that will cycle until a max number of steps (i.e., segments) has been produced. The first step will have the LLM create a new `Segment`, which will be used to create a new story `Block`. The second step will prompt the human to choose their adventure from the list of actions specified in the newly created `Segment`.

In [16]:
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

In [17]:
class NewBlockEvent(Event):
    block: Block


class HumanChoiceEvent(Event):
    block_id: str

In [18]:
class ChooseYourOwnAdventureWorkflow(Workflow):
    def __init__(self, max_steps: int = 3, **kwargs):
        super().__init__(**kwargs)
        self.llm = OpenAI("gpt-4o-mini")
        self.max_steps = max_steps

    @step
    async def create_segment(
        self, ctx: Context, ev: StartEvent | HumanChoiceEvent
    ) -> NewBlockEvent | StopEvent:
        blocks = await ctx.get("blocks", [])
        running_story = "\n".join(str(b) for b in blocks)

        if len(blocks) < self.max_steps:
            new_segment = self.llm.structured_predict(
                Segment,
                PromptTemplate(SEGMENT_GENERATION_TEMPLATE),
                running_story=running_story,
            )
            new_block = Block(segment=new_segment)
            blocks.append(new_block)
            await ctx.set("blocks", blocks)
            return NewBlockEvent(block=new_block)
        else:
            final_segment = self.llm.structured_predict(
                Segment,
                PromptTemplate(FINAL_SEGMENT_GENERATION_TEMPLATE),
                running_story=running_story,
            )
            final_block = Block(segment=final_segment)
            blocks.append(final_block)
            return StopEvent(result=blocks)

    @step
    async def prompt_human(
        self, ctx: Context, ev: NewBlockEvent
    ) -> HumanChoiceEvent:
        block = ev.block

        # get human input
        human_prompt = f"\n===\n{ev.block.segment.plot}\n\n"
        human_prompt += "Choose your adventure:\n\n"
        human_prompt += "\n".join(ev.block.segment.actions)
        human_prompt += "\n\n"
        human_input = input(human_prompt)

        blocks = await ctx.get("blocks")
        block.choice = human_input
        blocks[-1] = block
        await ctx.set("block", blocks)

        return HumanChoiceEvent(block_id=ev.block.id_)

### Running The Workflow

Since workflows are async first, this all runs fine in a notebook. If you were running in your own code, you would want to use `asyncio.run()` to start an async event loop if one isn't already running.

```python
async def main():
    <async code>

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

In [19]:
import nest_asyncio

nest_asyncio.apply()

In [20]:
w = ChooseYourOwnAdventureWorkflow(timeout=None)

In [21]:
result = await w.run()


===
In a small village nestled between towering mountains, a young woman named Elara discovers an ancient map hidden in her grandmother's attic. The map hints at a legendary treasure buried deep within the Forbidden Forest, a place rumored to be cursed. Driven by curiosity and the desire to change her family's fortunes, Elara decides to embark on a quest to find the treasure, but she must first gather supplies and seek advice from the village elder.

Choose your adventure:

Visit the village market to gather supplies
Consult the village elder for advice
Prepare a plan for the journey into the Forbidden Forest

Consult the village elder for advice

===
Elara approaches the village elder, a wise woman named Maelis, who has lived for many years and knows the secrets of the Forbidden Forest. Maelis warns Elara about the dangers that lie ahead, including mythical creatures and treacherous terrain, but she also reveals that the treasure is protected by a riddle that must be solved. To aid E

### Print The Final Story

In [22]:
final_story = "\n\n".join(b.segment.plot for b in result)
print(final_story)

In a small village nestled between towering mountains, a young woman named Elara discovers an ancient map hidden in her grandmother's attic. The map hints at a legendary treasure buried deep within the Forbidden Forest, a place rumored to be cursed. Driven by curiosity and the desire to change her family's fortunes, Elara decides to embark on a quest to find the treasure, but she must first gather supplies and seek advice from the village elder.

Elara approaches the village elder, a wise woman named Maelis, who has lived for many years and knows the secrets of the Forbidden Forest. Maelis warns Elara about the dangers that lie ahead, including mythical creatures and treacherous terrain, but she also reveals that the treasure is protected by a riddle that must be solved. To aid Elara, Maelis offers her a magical amulet that can guide her through the forest, but warns that it can only be used once.

Maelis explains that the riddle is inscribed on a stone tablet located at the heart of t

### Other Ways To Implement Human In The Loop

One could also implement the human in the loop by creating a separate Workflow just for gathering human input and making use of nested Workflows. This design could be used in situations where you would want the human input gathering to be a separate service from the rest of the Workflow, which is what would happen if you deployed the nested workflows with llama-deploy.